In [ ]:
import requests as rq
import bs4 as bs4
import time
import json
import pandas as pd
import numpy as np
import math
import time

# Parte 1: Editoras

In [ ]:
# Url onde estão todas as editoras

url_editoras = 'https://www.skoob.com.br/editoras/'

In [ ]:
# Fazendo request do link e transformando em um objeto do bs4

response = rq.get(url_editoras)
pagina_html = bs4.BeautifulSoup(response.text)

In [ ]:
# Procurando tags

tags = pagina_html.findAll(id="resultadoBusca-")
editoras = tags[0].findAll('strong')

lista_editoras = []
lista_links = []

for x in editoras:
    if x.find('a') != None:
        editora = x.get_text()
        lista_editoras.append(editora)
        
        link = x.find('a').get('href')
        lista_links.append(link)
        
livros_editora = []
numero_livros = tags[0].findAll('span')

for livros in range(1, len(numero_livros), 8):
    n_livros = numero_livros[livros].get_text().split()[0]
    n_livros = n_livros.replace('.', '')
    livros_editora.append(n_livros)

In [ ]:
# Criando dataframe que armazenará os dados

df = pd.DataFrame()
df['editora'] = lista_editoras
df['link'] = lista_links
df['qtde_livros'] = livros_editora

In [ ]:
# Função para ser aplicada em cada linha do dataframe na coluna código
# Parâmetro x: string -> link da editora
# Irá retornar o código da editora

def codigo(x):
    return x.split('/')[-1].split('-')[0]

codigos = df_editoras['link'].apply(codigo)
df['codigo'] = codigos

In [ ]:
# Dataframe com as editoras
df.to_csv('./editoras_link.csv', index=False)

In [ ]:
df

# Parte 2: Livros das Editoras


Os livros de cada editora estão no link : "https://www.skoob.com.br/editora/livros/codigoDeCadaEditora"  
É necessário o código de cada editora para entrar na página dos livros. Os códigos são os números da coluna link do df_editoras

In [ ]:
lista_livros = []
lista_livros_link = []

for index, row in df.iterrows():
    print('--------------------------')
    print(df['editora'][index])
    
    
    url_base = 'https://www.skoob.com.br/editora/livros/{}/mpage:{}'
    
    # Número de páginas com livros 
    n_page = math.ceil(int(df['qtde_livros'][index])/40) + 1
    
    for x in range(1, n_page):
        print(x)
        url_livros = url_base.format(df['codigo'][index], x)
        
        response = rq.get(url_livros)
        pagina_html = bs4.BeautifulSoup(response.text)
        
        lista_div = pagina_html.findAll("div", {"class": "box_livro"})
        
        for x in lista_div:
            titulo = x.find('h3').get_text()
            link = x.find('a').get('href')
            
            lista_livros.append(titulo)
            lista_livros_link.append(link)
        
        time.sleep(1)


In [ ]:
len(lista_livros), len(lista_livros_link)

In [ ]:
df = pd.DataFrame()
df['livros'] = lista_livros
df['link'] = lista_livros_link

In [ ]:
# Dataframe com as editoras
df.to_csv('./livros_link.csv', index=False)

In [ ]:
df

# Livros

In [ ]:
df = pd.read_csv('./livros_link.csv')

In [ ]:
info_livros = []

for index, row in df.iterrows():
    
    livro = {}
    
    if index % 10 == 0:
        time.sleep(1)
    try:
        # Request para a paǵina do livro
        url_base = 'https://www.skoob.com.br{}'.format(df['link'][index])        
        response = rq.get(url_base)
        pagina_html = bs4.BeautifulSoup(response.text)

        div_lateral = pagina_html.find("div", {"id": "pg-livro-menu-principal"})
        titulo = div_lateral.find('strong').get_text()
        livro['titulo'] = titulo

        print(f'{index} - {titulo}')
        print(url_base)

        autor = div_lateral.findAll('a')[1].get_text()
        livro['autor'] = autor

        div_menor = div_lateral.find('div', {"class": "sidebar-desc"})
        isbns = div_menor.findAll('span')
        try:
            livro['ISBN_13'] = str(isbns[0]).split('>')[1].split('<')[0]
            livro['ISBN_10'] = str(isbns[1]).split('>')[1].split('<')[0]
        except:
            livro['ISBN_13'] = ''
            livro['ISBN_10'] = ''
            
        
        texto = div_menor.get_text()
        
        try:    
            livro['ano'] = texto.split('Ano: ')[1].split(' ')[0]
        except:
            livro['ano'] = ''
        
        try:
            livro['paginas'] = texto.split('Páginas: ')[1].split(' ')[0]
        except:
            livro['paginas'] = ''
            
        try:
            livro['idioma'] = texto.split('Idioma: ')[1].split(' ')[0]
        except:
            livro['idioma'] = ''
        
        try:
            livro['editora'] = texto.split('Editora: ')[1].split(' ')[0]
        except:
            livro['editora'] = ''
            

        div_superior = pagina_html.find('div', {'id': 'livro-perfil-status'})
        
        try:
             livro['rating'] = div_superior.find('span', {'class': 'rating'}).get_text()
        except:
             livro['rating'] = ''
        
        try:
            livro['avaliacao'] = div_superior.find('div', {'id': 'pg-livro-box-rating-avaliadores-numero'}).get_text().split(' ')[0]
        except:
            livro['avaliacao'] = ''


        info = div_superior.findAll('a', {'class': 'text_blue'})
        
        try:
            livro['resenha'] = info[0].get_text()
        except:
            livro['resenha'] = ''
            
        try:
            livro['abandonos']  = info[1].get_text()
        except:
            livro['abandonos'] = ''
        
        try:
            livro['relendo']  = info[2].get_text()
        except:
            livro['relendo'] = ''
            
        try:   
            livro['querem_ler'] = info[3].get_text()
        except:
            livro['querem_ler'] = ''
        
        try:
            livro['lendo'] = info[4].get_text()
        except:
            livro['lendo'] = ''
            
        try:
            livro['leram'] = info[5].get_text()
        except:
            livro['leram'] = ''


        info = pagina_html.find('p', {'itemprop': 'description'})
        
        try: 
            descricao = info.get_text().replace('\n', '')
            descricao = descricao.split('.')
            genero = descricao[-1]
            descricao = ''.join(descricao[:-1])
            livro['descricao'] = descricao
            livro['genero'] = genero
        except:
            livro['descricao'] = ''
            livro['genero'] = ''

        try:
            info = pagina_html.find_all('div', {'id': 'livro-perfil-resenhas'})[0]
            qte_resenhas = info.find('h2', {'class': 'secao-subtitulo'}).get_text()
            livro['qte_resenhas'] = qte_resenhas.split('(')[-1].split(')')[0]
        except:
            livro['qte_resenhas'] = 0
            
        
        try: 
            info = pagina_html.find('span', {'class': 'pg-livro-icone-male-label'})
            livro['male'] = int(info.get_text().replace('%', ''))

            info = pagina_html.find('span', {'class': 'pg-livro-icone-female-label'})
            livro['female'] = int(info.get_text().replace('%', ''))
        
        except: 
            livro['male'] = 0
            livro['female'] = 0
            
        info_livros.append(livro)
        
    except Exception as e:
        print(e)
        continue

In [ ]:
dd = pd.DataFrame(info_livros)
dd.to_csv('./dados.csv')

In [ ]:
dd[dd['autor'] == '\nR$ \n']

In [ ]:
for index, row in dd.iterrows():
    
    if dd['autor'][index] == '\nR$ \n':
        
        livro = dd['titulo'][index]
        link = df[df['livros'] == livro].reset_index()
        try:
            url_base = 'https://www.skoob.com.br{}'.format(link['link'][0])
            response = rq.get(url_base)
            pagina_html = bs4.BeautifulSoup(response.text)
        except:
            continue
        try:
            autor = pagina_html.find("i", {"class": "sidebar-subtitulo"}).get_text()
        except:
            autor = ''
            print(url_base)
            
        dd.at[index, 'autor'] = autor
        print(autor)


In [13]:
dd

NameError: name 'dd' is not defined

In [ ]:
df[df['livros'] == livro], livro

In [ ]:
dd['autor'][index] == '\nR$ \n'

In [ ]:
df = pd.read_csv('./dados_novos.csv')

In [ ]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [12]:
str(int(df['ISBN_13'][123]))

'9788551003718'

In [ ]:
dd[dd['autor'] == '\nR$ \n']

In [ ]:
dd.head()